# Data Grouping

## 📘 Cell 1: Import dan Setup Folder

In [20]:
# 📦 Import library yang dibutuhkan
import os
import pandas as pd

## 📘 Cell 2: Definisikan Folder dan Inisialisasi

In [21]:
# 📁 Path ke folder utama
root_folder = "DataGrouping1"

# 📂 Inisialisasi struktur data untuk semua tipe model
all_data = {
    "ensemble": [],
    "konvensional": [],
    "ann": [],
    "kfold": []
}

## 📘 Cell 3: Membaca Semua CSV dan Normalisasi

In [22]:
# 🔁 Membaca semua file CSV dalam subfolder dan normalisasi kolom
for subdir in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subdir)

    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)

            if file.endswith(".csv"):
                df = pd.read_csv(file_path)

                # --- Untuk file KFold ---
                if "kfold" in file.lower():
                    # Ambil kolom penting dan rename agar konsisten
                    df_kfold = df[[
                        "Percobaan", "Model",
                        "Accuracy_Test", "Precision_Test", "Recall_Test", "F1-Score_Test"
                    ]].copy()

                    df_kfold = df_kfold.rename(columns={
                        "Accuracy_Test": "Accuracy",
                        "Precision_Test": "Precision",
                        "Recall_Test": "Recall",
                        "F1-Score_Test": "F1-Score"
                    })

                    df_kfold["Tipe"] = "kfold"
                    df_kfold["Sumber"] = subdir
                    all_data["kfold"].append(df_kfold)

                # --- Untuk file ensemble ---
                elif "ensemble" in file.lower():
                    df["Tipe"] = "ensemble"
                    df["Sumber"] = subdir
                    all_data["ensemble"].append(df)

                # --- Untuk file ann ---
                elif "ann" in file.lower():
                    df["Tipe"] = "ann"
                    df["Sumber"] = subdir
                    all_data["ann"].append(df)

                # --- Untuk file model konvensional ---
                elif "konvensional" in file.lower():
                    df["Tipe"] = "konvensional"
                    df["Sumber"] = subdir
                    all_data["konvensional"].append(df)

## 📘 Cell 4: Gabungkan Semua Data

In [23]:
# 📊 Gabungkan semua data menjadi satu DataFrame
df_all = pd.concat(
    all_data["ensemble"] +
    all_data["konvensional"] +
    all_data["ann"] +
    all_data["kfold"],
    ignore_index=True
)

# Pastikan kolom evaluasi dalam format numerik
for metric in ["Accuracy", "Precision", "Recall", "F1-Score"]:
    df_all[metric] = pd.to_numeric(df_all[metric], errors="coerce")

## 📘 Cell 5: Fungsi untuk Ambil Model Terbaik/Terburuk

In [ ]:
# # 🎯 Fungsi untuk mengambil n model terbaik atau terburuk berdasarkan F1-Score
# def get_top_bottom(df, tipe=None, top_n=10, worst=False):
#     temp = df.copy()
#     if tipe:
#         temp = temp[temp["Tipe"] == tipe]
    
#     # Urutkan berdasarkan F1-Score
#     temp = temp.sort_values("F1-Score", ascending=worst)
#     return temp.head(top_n)

# # 🎯 Fungsi untuk mengambil n model terbaik atau terburuk berdasarkan F1-Score
# def get_top_bottom(df, tipe=None, top_n=10, worst=False):
#     temp = df.copy()
#     if tipe:
#         temp = temp[temp["Tipe"] == tipe]
#     # 🔑 Urutan sorting multi-kriteria
#     temp = temp.sort_values(
#         ["F1-Score", "Precision", "Recall", "Accuracy"],
#         ascending=[worst, worst, worst, worst]
#     )
#     return temp.head(top_n)

# 🎯 Fungsi untuk mengambil n model terbaik atau terburuk berdasarkan F1-Score
def get_top_bottom(df, tipe=None, top_n=10, worst=False):
    temp = df.copy()
    if tipe:
        temp = temp[temp["Tipe"] == tipe]
    # 🔑 Urutan sorting multi-kriteria
    temp = temp.sort_values(
        ["F1-Score", "Recall", "Precision", "Accuracy"],
        ascending=[worst, worst, worst, worst]
    )
    return temp.head(top_n)


## 📘 Cell 6: Ekstrak Data Terbaik & Terburuk

In [25]:
# 🏆 Ekstrak hasil terbaik & terburuk sesuai task
top_20_models = get_top_bottom(df_all, top_n=20)
top_10_ensemble = get_top_bottom(df_all, tipe="ensemble", top_n=10)
top_10_ann = get_top_bottom(df_all, tipe="ann", top_n=10)
bottom_20_models = get_top_bottom(df_all, top_n=20, worst=True)
bottom_10_ensemble = get_top_bottom(df_all, tipe="ensemble", top_n=10, worst=True)

## 📘 Cell 7: Simpan ke Excel

In [ ]:
# 💾 Simpan semua hasil ke file Excel
output_dir = "ResultGrouping"
os.makedirs(output_dir, exist_ok=True)  # Membuat folder jika belum ada

output_file_full = os.path.join(output_dir, "hasil_klasifikasi_model1_Ryan.xlsx")

with pd.ExcelWriter(output_file_full, engine='xlsxwriter') as writer:
    top_20_models.to_excel(writer, sheet_name="20_Model_Terbaik", index=False)
    top_10_ensemble.to_excel(writer, sheet_name="10_Ensemble_Terbaik", index=False)
    top_10_ann.to_excel(writer, sheet_name="10_ANN_Terbaik", index=False)
    bottom_20_models.to_excel(writer, sheet_name="20_Model_Terburuk", index=False)
    bottom_10_ensemble.to_excel(writer, sheet_name="10_Ensemble_Terburuk", index=False)

    workbook = writer.book
    format_full = workbook.add_format({'num_format': '0.000000000000000'})
    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]
        worksheet.set_column('C:F', 25, format_full)

print(f"File presisi penuh tersimpan: {output_file_full}")

File presisi penuh tersimpan: ResultGrouping\hasil_klasifikasi_model1.xlsx


## 📘 Cell 8: Simpan Excel – Ringkas 3 Desimal

In [ ]:
output_dir = "ResultGrouping"
os.makedirs(output_dir, exist_ok=True)  # Membuat folder jika belum ada

output_file_round = os.path.join(output_dir, "hasil_klasifikasi_model_ringkas1_Ryan.xlsx")

def round_df(df):
    df_rounded = df.copy()
    for metric in ["Accuracy", "Precision", "Recall", "F1-Score"]:
        df_rounded[metric] = df_rounded[metric].round(3)
    return df_rounded

with pd.ExcelWriter(output_file_round) as writer:
    round_df(top_20_models).to_excel(writer, sheet_name="20_Model_Terbaik", index=False)
    round_df(top_10_ensemble).to_excel(writer, sheet_name="10_Ensemble_Terbaik", index=False)
    round_df(top_10_ann).to_excel(writer, sheet_name="10_ANN_Terbaik", index=False)
    round_df(bottom_20_models).to_excel(writer, sheet_name="20_Model_Terburuk", index=False)
    round_df(bottom_10_ensemble).to_excel(writer, sheet_name="10_Ensemble_Terburuk", index=False)
    
    workbook = writer.book
    format_3 = workbook.add_format({'num_format': '0.000'})
    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]
        worksheet.set_column('C:F', 25, format_3)

print(f"✅ Disimpan versi ringkas ke: {output_file_round}")


✅ Disimpan versi ringkas ke: ResultGrouping\hasil_klasifikasi_model_ringkas1.xlsx
